In [ ]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from Bio import SeqIO
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import seaborn as sns
import random
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import matthews_corrcoef, make_scorer
from statistics import stdev, variance, mean
import torch
from torch import nn, optim
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from scipy.interpolate import interp1d
from sklearn.decomposition import PCA

In [ ]:
client_set=set()
scaffold_set=set()
others_set=set()
for rec in SeqIO.parse("../fig1/result/drllps_client_clstr_Homo_sapiens.fasta", "fasta"):
    client_set.add(rec.id)
for rec in SeqIO.parse("../fig1/result/drllps_scaffold_clstr_Homo_sapiens.fasta", "fasta"):
    scaffold_set.add(rec.id)
for rec in SeqIO.parse("../fig1/result/drllps_nonllps_clstr_Homo_sapiens.fasta", "fasta"):
    others_set.add(rec.id)
    
mat=np.load("../fig2/embedding/PTT5XLU50_human.npy", allow_pickle=True)
mat=mat.item()

list_client=[]
list_others=[]
list_scaffold=[]
client_id=[]
others_id=[]
scaffold_id=[]
for k in mat.keys():
    if k in others_set:
        list_others.append(mat[k])
        others_id.append(k)
    elif k in client_set:
        list_client.append(mat[k])
        client_id.append(k)
    elif k in scaffold_set:
        list_scaffold.append(mat[k])
        scaffold_id.append(k)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, path='checkpoint_model.pth'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:  
            self.best_score = score   
            self.checkpoint(val_loss, model)  
        elif score <= self.best_score:  
            self.counter += 1   
            if self.verbose:  
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')   
            if self.counter >= self.patience:  
                self.early_stop = True
        else:  
            self.best_score = score  
            self.checkpoint(val_loss, model)  
            self.counter = 0  
            
    def checkpoint(self, val_loss, model):
        if self.verbose:  
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)  
        self.val_loss_min = val_loss  
        
def training_loop(n_epochs, optimizer, model, loss, mask_train, x_train,  y_train):
    loss=loss
    
    n_samples=x_train.shape[0]
    n_val=int(n_samples*0.2)

    shuffled_ind=torch.randperm(n_samples)

    train_ind=shuffled_ind[:-n_val] 
    val_ind=shuffled_ind[-n_val:]
    
    x_val=x_train[val_ind]
    y_val=y_train[val_ind]
    
    x_train=x_train[train_ind]
    y_train=y_train[train_ind]
    
    x_train=x_train
    y_train=y_train
    
    x_val=x_val
    y_val=y_val

    patience=10
    earlystopping = EarlyStopping(patience=patience, verbose=False)
    for epoch in range(1, n_epochs+1):
        model.train()
        
        y_train_pred=model.forward(x_train)
        loss_train=loss(y_train_pred, y_train)
        
        model.eval()
        with torch.no_grad():
            y_val_pred=model.forward(x_val)
            loss_val=loss(y_val_pred, y_val)

        earlystopping(loss_val, model) 
        if earlystopping.early_stop: 
            break
            
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
class FNN2(nn.Module):
    def __init__(self, embeddings_dim=1024, dropout=0.25):
        super(FNN2, self).__init__()

        self.linear = nn.Sequential(
            nn.Linear(embeddings_dim, 32),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(32,2)
        )


    def forward(self, x: torch.Tensor, **kwargs) -> torch.Tensor:
        o = self.linear(x)  
        return o
    

class NN(BaseEstimator, ClassifierMixin):
    def __init__(self, n_epochs=500, lr=0.03):
        self.n_epochs = n_epochs
        self.lr = lr
        self.model = None
        self.optim = None
        self.loss = nn.CrossEntropyLoss()

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        X_tensor = torch.tensor(X, dtype=torch.float)
        y_tensor = torch.tensor(y, dtype=torch.long)
        n_dim = X_tensor.shape[1]
        self.model=FNN2(embeddings_dim=n_dim)
        self.optim = optim.Adam(self.model.parameters(), lr=self.lr)
        training_loop(
            n_epochs=self.n_epochs,
            optimizer=self.optim,
            model=self.model,
            loss=self.loss,
            mask_train=None,
            x_train=X_tensor,
            y_train=y_tensor,
        )
        return self

    def predict(self, X):
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float)
            self.model.eval()
            y_pred = self.model(X_tensor)
            _, predicted = torch.max(y_pred, 1)
            return predicted.numpy()

    def predict_proba(self, X):
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float)
            self.model.eval()
            y_pred = self.model(X_tensor)
            probas = nn.Softmax(dim=1)(y_pred)
            return probas.numpy()

In [ ]:
def under_sampling(x, y, idx):
    x_ture=x[y==True]
    x_false=x[y==False]
    y_ture=y[y==True]
    y_false=y[y==False]
    idx_ture=idx[y==True]
    idx_false=idx[y==False]
    positive_n=len(y_ture)
    negative_n=len(y_false)
    random_index=np.random.randint(0,negative_n,positive_n)  
    x_false_u=x_false[random_index]
    y_false_u=y_false[random_index]
    idx_false_u=idx_false[random_index]
    return np.concatenate([x_ture, x_false_u]), np.concatenate([y_ture, y_false_u]), np.concatenate([idx_ture, idx_false_u])

In [ ]:
np.random.seed(0)
x_all=np.array(list_client+list_others)
y_all=np.array([True]*len(list_client) + [False]*len(list_others))
idx_all=np.array(client_id+others_id)
x,y,idx=under_sampling(x_all,y_all,idx_all)
estimators = [
    ('nn', NN(lr=0.01)),
    ('rf', RandomForestClassifier(max_depth=20, max_features="sqrt", class_weight="balanced",n_estimators=200, n_jobs=40)),
    ('svm', make_pipeline(StandardScaler(), SVC(class_weight="balanced", probability=True, gamma="auto"))),
    ('hgboost', HistGradientBoostingClassifier(learning_rate=0.1, max_leaf_nodes=63, min_samples_leaf=80, class_weight="balanced"))
]
cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
model=StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(class_weight="balanced"), n_jobs=-1, cv=cv
)

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
for train, test in skf.split(x,y):
    model.fit(x[train], y[train])
    
    id_train=idx[train]
    x_test=x[test]
    y_test=y[test]
    idx_test=idx[test]
    x_test_t=x_test[y_test==True]
    idx_test_t=idx_test[y_test==True]
    
    non_no_train=others_set - set(id_train)
    x_no_train_f=[]
    idx_no_tran_f=[]
    for k in mat.keys():
        if k in non_no_train:
            x_no_train_f.append(mat[k])
            idx_no_tran_f.append(k)
    x_no_train_f=np.array(x_no_train_f)
    
    y_test_t_pred=model.predict_proba(x_test_t)
    y_no_train_f_pred=model.predict_proba(x_no_train_f)
    break

In [ ]:
f_pred_sorted_idx=np.argsort(y_no_train_f_pred[:,1])[::-1]
t_pred_sorted_idx=np.argsort(y_test_t_pred[:,1])[::-1]
non_id_sorted=np.array(idx_no_tran_f)[f_pred_sorted_idx]
cli_id_sorted=idx_test_t[t_pred_sorted_idx]

In [ ]:
non_id_sorted=[i.split("|")[1] for i in non_id_sorted]
cli_id_sorted=[i.split("|")[1] for i in cli_id_sorted]

In [ ]:
non_id_sorted[:10]

In [ ]:
cli_id_sorted[:10]

In [ ]:
mat=np.load("data/PTT5XLU50_human_aa.npy", allow_pickle=True)
mat=mat.item()

top_n=20

client_like_client_set=set(cli_id_sorted[:top_n])
client_like_nonllps_set=set(non_id_sorted[:top_n])

list_client_mat_d={}
list_non_mat_d={}
client_like_client_d={}
client_like_nonllps_d={}

for k in mat.keys():
    k_id=k.split("|")[1]
    if k_id in client_like_client_set:
        list_client_mat_d[k_id]=mat[k]
        client_like_client_d[k_id]=k_id
    elif k_id in client_like_nonllps_set:
        list_non_mat_d[k_id]=mat[k]
        client_like_nonllps_d[k_id]=k_id
        
list_client_mat=[list_client_mat_d[k_id] for k_id in cli_id_sorted[:top_n]]
client_like_client=[client_like_client_d[k_id] for k_id in cli_id_sorted[:top_n]]
list_non_mat=[list_non_mat_d[k_id] for k_id in non_id_sorted[:top_n]]
client_like_nonllps=[client_like_nonllps_d[k_id] for k_id in non_id_sorted[:top_n]]
del mat

In [ ]:
def sliding_window_avg(arr, window_size):
    if len(arr.shape) != 2:
        raise ValueError("Input must be a two-dimensional array")

    if window_size > arr.shape[0]:
        raise ValueError("Window size cannot be larger than the array length")

    result = np.zeros_like(arr)
    pad_size = window_size // 2
    for i in range(arr.shape[0]):
        start = max(0, i - pad_size)
        end = min(arr.shape[0], i + pad_size + 1)
        result[i] = np.mean(arr[start:end], axis=0)
    return result

In [ ]:
y_arrs_client={}
for mat, protein_id in zip(list_client_mat, client_like_client):
    x_arr=sliding_window_avg(mat, 100)
    y_arr=model.predict_proba(x_arr)
    y_arrs_client[protein_id]=y_arr[:,1]
    plt.axhline(y=0.5, linestyle='dashed', color='grey')
    plt.plot(y_arr[:,1])
    plt.ylim([0.2,0.8])
    plt.ylabel("Client score")
    plt.xlabel("Sequence")
    plt.title(protein_id)
    plt.show()

In [ ]:
y_arrs_nonllps={}
for mat, protein_id in zip(list_non_mat, client_like_nonllps):
    x_arr=sliding_window_avg(mat, 100)
    y_arr=model.predict_proba(x_arr)
    y_arrs_nonllps[protein_id]=y_arr[:,1]
    plt.axhline(y=0.5, linestyle='dashed', color='grey')
    plt.plot(y_arr[:,1])
    plt.ylim([0.2,0.8])
    plt.ylabel("Client score")
    plt.xlabel("Sequence")
    plt.title(protein_id)
    plt.show()

In [ ]:
sorted_list_client = sorted(y_arrs_client.items(), key=lambda x: np.mean(x[1] < 0.5), reverse=True)
sorted_list_nonllps = sorted(y_arrs_nonllps.items(), key=lambda x: np.mean(x[1] < 0.5), reverse=True)

In [ ]:
for x in sorted_list_client[:3]:
    plt.axhline(y=0.5, linestyle='dashed', color='grey')
    plt.plot(x[1])
    plt.ylim([0.25,0.85])
    plt.ylabel("Client score")
    plt.xlabel("Sequence")
    plt.title(x[0])
    plt.savefig("result/fig7_"+x[0]+".pdf")
    plt.show()

In [ ]:
for x in sorted_list_nonllps[:3]:
    plt.axhline(y=0.5, linestyle='dashed', color='grey')
    plt.plot(x[1])
    plt.ylim([0.25,0.85])
    plt.ylabel("Client score")
    plt.xlabel("Sequence")
    plt.title(x[0])
    plt.savefig("result/fig7_"+x[0]+".pdf")
    plt.show()

In [ ]:
for x in sorted_list_client[:3]:
    indices = np.where(x[1] >= 0.5)
    indices_1=[i+1 for i in indices[0]]
    start=indices_1[0]
    j=indices_1[0]
    s=str(start)
    for i in indices_1:
        if i == start:
            continue
        if j+1 == i:
            j=i
            continue
        elif start==j:
            s=s+"+"+str(i)
            start=i
            j=i
        else:
            s=s+"-"+str(j)+"+"+str(i)
            start=i
            j=i
    if start!=j:
        s=s+"-"+str(j)
    print(x[0])
    print(s)

In [ ]:
for x in sorted_list_nonllps[:3]:
    indices = np.where(x[1] >= 0.5)
    indices_1=[i+1 for i in indices[0]]
    start=indices_1[0]
    j=indices_1[0]
    s=str(start)
    for i in indices_1:
        if i == start:
            continue
        if j+1 == i:
            j=i
            continue
        elif start==j:
            s=s+"+"+str(i)
            start=i
            j=i
        else:
            s=s+"-"+str(j)+"+"+str(i)
            start=i
            j=i
    if start!=j:
        s=s+"-"+str(j)
    print(x[0])
    print(s)